In [5]:
# from fastFM import als 
import pandas as pd
from numpy.linalg import norm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error,recall_score,ndcg_score
from sklearn.model_selection import ShuffleSplit
from scipy.sparse import csc_matrix

'''
https://github.com/ibayer/fastFM
'''

'\nhttps://github.com/ibayer/fastFM\n'

In [6]:
import pywFM
import numpy as np
import pandas as pd

In [25]:
movie_genre = np.load('../hw3_data/Movielens/movie_genre.npy')
movie_movie = np.load('../hw3_data/Movielens/movie_movie.npy')
user_age = np.load('../hw3_data/Movielens/user_age.npy')
user_movie = np.load('../hw3_data/Movielens/user_movie.npy')
user_occupation = np.load('../hw3_data/Movielens/user_occupation.npy')
user_user = np.load('../hw3_data/Movielens/user_user.npy')
entity_user = 943
entity_item = 1682
age, occupation, genre = 8, 21, 18

In [20]:
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)
for i, (train_index, test_index) in enumerate(rs.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

In [45]:
user,item = X_train[0]
user,item

(627, 649)

In [48]:
fea_age = np.zeros((X_train.shape[0], age), dtype=int)
fea_occupation = np.zeros((X_train.shape[0], occupation), dtype=int)
fea_genre = np.zeros((X_train.shape[0], genre), dtype=int)

In [49]:
ua = user_age[user_age[:,0] == user][:,1]
fea_age[0,ua-1] = 1

uo = user_occupation[user_occupation[:,0] == user][:,1]
fea_occupation[0,uo-1] = 1

mg = movie_genre[movie_genre[:,0] == item][:,1]
fea_genre[0,mg-1] = 1